##Set up


In [1]:
# lots of imports for now. Will strip it down later

import sys
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, plot_roc_curve, roc_auc_score, plot_confusion_matrix
from sklearn.inspection import permutation_importance

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
# import spotipy.util as util

#from google.colab import files
#from google.colab import drive
#drive.mount('drive', force_remount=True)

In [2]:
# data_path references data.csv from our github repo
data_path = 'https://raw.githubusercontent.com/StephenSpicer/Spotify_Music_Discovery_LS_DS_BW/main/data/data.csv'

## Wrangle Data


In [3]:
# Very basic function. Need to add new features if necessary.

# Data Wrangle function:


def wrangle(filepath):
  df = pd.read_csv(filepath, parse_dates=['release_date'])

  # fixing artist column by stripping  [   ]  '  and  "
  df['artists'] = df['artists'].str.strip("[]")
  df['artists'] = df['artists'].str.strip("'")
  df['artists'] = df['artists'].str.strip('"')
  df['artists'] = df['artists'].str.lower()
  df['name'] = df['name'].str.lower()

  # make new features
  df['month'] = df['release_date'].dt.month

  # set release date as the index
  df.set_index('id', inplace=True)
  
  # rearraging similar features to be together to improve readability
  df = df[['name', 'artists', 'valence', 'acousticness', 'danceability',
        'energy', 'instrumentalness', 'liveness', 'loudness',  
        'speechiness','tempo'
        ]]
  df['loudness'] = df.loudness.apply(lambda x: x / -100)
  df['tempo'] = df.tempo.apply(lambda x: x / 1000)
  df = df[df.loudness > 0.0]
  df = df.drop(['name', 'artists'], axis=1)         
  return df

def the_key(filepath):
  df = pd.read_csv(filepath, parse_dates=['release_date'])

  # fixing artist column by stripping  [   ]  '  and  "
  df['artists'] = df['artists'].str.strip("[]")
  df['artists'] = df['artists'].str.strip("'")
  df['artists'] = df['artists'].str.strip('"')
  df['artists'] = df['artists'].str.lower()
  df['name'] = df['name'].str.lower()

  # make new features
  df['month'] = df['release_date'].dt.month

  # set release date as the index
  df.set_index('id', inplace=True)

  # rearraging similar features to be together to improve readability
  df = df[['name', 'artists', 'valence', 'acousticness', 'danceability',
        'energy', 'instrumentalness', 'liveness', 'loudness',  
        'speechiness','tempo'
        ]]
  df['loudness'] = df.loudness.apply(lambda x: x / -100)
  df['tempo'] = df.tempo.apply(lambda x: x / 1000)
  df = df[df.loudness > 0.0]
  
  key = df[['name', 'artists']]

  return key

In [4]:
# read in the data and save it to df
df = wrangle(data_path)
df.head()

,valence,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,
0cS0A1fUEUd1EW3FcF8AEI,0.6340,0.991000,0.598,0.224,0.000522,0.3790,0.12628,0.0936,0.149976
0hbkKFIJm7Z05H8Zl9w30f,0.9500,0.643000,0.852,0.517,0.026400,0.0809,0.07261,0.0534,0.086889
11m7laMUgmOKqI3oYzuhne,0.6890,0.993000,0.647,0.186,0.000018,0.5190,0.12098,0.1740,0.097600
19Lc5SfJJ5O1oaxY0fpwfh,0.0422,0.000173,0.730,0.798,0.801000,0.1280,0.07311,0.0425,0.127997
2hJjbsLCytGsnAHfdsLejp,0.2990,0.295000,0.704,0.707,0.000246,0.4020,0.06036,0.0768,0.122076


In [5]:
key = the_key(data_path)
key.head()

,name,artists
id,,
0cS0A1fUEUd1EW3FcF8AEI,keep a song in your soul,mamie smith
0hbkKFIJm7Z05H8Zl9w30f,i put a spell on you,screamin' jay hawkins
11m7laMUgmOKqI3oYzuhne,golfing papa,mamie smith
19Lc5SfJJ5O1oaxY0fpwfh,true house music - xavier santos & carlos gomi...,oscar velazquez
2hJjbsLCytGsnAHfdsLejp,xuniverxe,mixe


### significance testing


In [6]:
# read in the data and save it to df
df = wrangle(data_path)

# Null hypothesis - target has a correlation with the selected metric. If the 

#year and month are not metrics we want our model to be trained on 
df = df.drop(['year', 'month', 'name'], axis=1)
#let's find out what key has significance with
target = 'key'
check = 

def significance(x):
  temp = []
  target = ['valence', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'key', 'mode']
  y = df[target]
  for i in x:

y = df[target]
x = df[['valence', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'key', 'mode']]
cutoff = 0.5
mask = x< cutoff
x_train = x.loc[mask]
y_train = y.loc[mask]
x_val, y_val = x.loc[~mask], y.loc[~mask]

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))
print(y_train.mean())


SyntaxError: invalid syntax (<ipython-input-6-307790f34f52>, line 10)

In [ ]:
#it makes sense for the MAE to be close to 0 on a scale of 1-5, let's see if we can find out what key is derived from
l1_model = LinearRegression()
l1_model.fit(x,y)
y_pred = l1_model.predict(x)
print('train MAE', mean_absolute_error(y, y_pred))

Here we see that our baseline and our training is very similar. Going forward we can test any column to see if it shows a significant difference between these two. If not then that column probably won't have a high vector value; which we can check later

In [ ]:
df = wrangle(data_path)
target = 'key'
y = df[target]
df = df[['popularity']]
l1_model = LinearRegression()
l1_model.fit(x,y)
y_pred = l1_model.predict(x)
print('train MAE', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'key'
y = df[target]
df = df[['popularity']]
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'key'
y = df[target]
df = df[['valence']]
l1_model = LinearRegression()
l1_model.fit(x,y)
y_pred = l1_model.predict(x)
print('train MAE', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'key'
y = df[target]
df = df[['valence']]
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

Key is doesn't seem to diverge based on any used, I am going to delete the 9 other boxes that relate to key, and note here that the MAE didn't change throughout. 

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df[['instrumentalness']]
l1_model = LinearRegression()
l1_model.fit(x,y)
y_pred = l1_model.predict(x)
print('train MAE', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df[['speechiness']]
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

Here we can see that acousticness has virtually 0 difference from instrumentalness, and a significant difference from speechiness. That being said, it is not the 3.05(06) that key had.

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df[['popularity']]
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df.valence
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df.danceability
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df.energy
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df.liveness
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df[['speechiness']]
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df.loudness
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df.tempo
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df.key
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

Key shouldn't have 

In [ ]:
df = wrangle(data_path)
target = 'acousticness'
y = df[target]
df = df[['speechiness']]
y_pred = [y.mean()]*len(y)
print('Baseline MAE:', mean_absolute_error(y, y_pred))

## EDA


## Split Data


In [10]:
# i used name as the target
df = wrangle(data_path)
x = df

### finding a solution to the loudness "problem" 


Above I divided loudness by 100 to get it between -1 and 1, I divided tempo by 1000 even though tempo should stop at >200 bmp. Maybe it matters that it's 5x smaller than it could be, but I don't think so. In order for us to normalize the data I first tried to figure out how many values of loudness were above and below 0. Turns out there were only 35 above 0 out of 178000 so i divided the entire column by -100 instead of 100 and then set the value of the column to > 0.0. 

In [11]:
# lx = x
# lx = lx.set_index('loudness')
# lx.head()

In [12]:
# cutoff = 0.0
# mask = lx.index > cutoff
# loud = lx.iloc[mask] 
# quiet = lx.iloc[~mask]
# #print(quiet.count())
# print(loud.count())


## Establish Baseline

##Build Model

In [13]:
def model(x):
  model = NearestNeighbors(n_neighbors=10,
                         n_jobs = -1)
  knn_spotify = model.fit(x)
  return knn_spotify

In [20]:
knn_spotify = model(x)

##Link song name + artist to model


In [14]:
selected_song = key.loc[key['name'].str.contains("golfing papa")]
selected_song
song_id = selected_song.index.tolist()
song_id

['11m7laMUgmOKqI3oYzuhne']

In [22]:
song_row = x.loc[song_id, :]
song_row

,valence,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
id,,,,,,,,,
11m7laMUgmOKqI3oYzuhne,0.689,0.993,0.647,0.186,0.000018,0.519,0.12098,0.174,0.0976


In [16]:
model(x)

NearestNeighbors(n_jobs=-1, n_neighbors=10)

In [17]:
neigh_dist, neigh_index = knn_spotify.kneighbors(song_row)

NameError: name 'knn_spotify' is not defined

In [ ]:
neigh_dist

In [ ]:
print(neigh_index)

In [ ]:
key.iloc[79559]

In [ ]:
x.iloc[79559]
#2l4Nmf7uR3wl4FZ6rre0Pp

In [ ]:
song_suggest = x.iloc[79559]
song_suggest


##Spotify API stuff


In [ ]:
client_id = 'id'
client_secret = 'secret'

scope = 'user-library-read'
if len(sys.argv) > 1:
  username = sys.argv[1]
  else:
    print("Usage: %s username" % (sys.argv[0]))
    sys.exit()

In [ ]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret = client_secret, redirect_uri='http://localhost:8881/')
sp = spotipy.Spotify(auth=token)

In [ ]:
#importing images for songs from api
  id_name = {}
  list_photo = {}
  for i in sp.current_user_playlists()['items']:
    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']
  #the app below needs to be tied into the spotify app above
  def vis_songs(something):
    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    for i, url in enumerate(temp):
      plt.subplot(len(temp) / columns + 1, columns, i +1)
      image = io.imread(url)
      plt.imshow(image)
      plt.xticks(color = 'w', fontsize = 0.1)
      plt.yticks(color = 'w', fontsize = 0.1)
      plt.xlabel(df['name'].values[i], fontsize = 12)
      plt.tight_layout(h_pad=0.4, w_pad=0)
      plt.subplots_adjust(wspace=None, hspace=None)

    return plt.show()

##Main function for converting ID


In [24]:
def id_grab(title, artist):
  key = the_key(data_path)
  #input title or title+artist
  if title and artist:
    selected_artist = key.loc[key['artists'].str.contains(artist)]
    selected_song = selected_artist.loc[selected_artist['name'].str.contains(title)]
  else:
    selected_song = key.loc[key['name'].str.contains(title)]

  #search the key df and return the song id
  song_id = selected_song.index.tolist()

  #feed the song id into the model
  song_row = x.loc[song_id, :]

  #model finds the NN and gives you back song id
  neigh_dist, neigh_index = knn_spotify.kneighbors(song_row)

  #random nn
  index = neigh_index.flat[0:10].tolist()
  song_index = random.choice(index)

  #convert the index of the suggested song to the song_id
  #This code is returning the column headers, not the index atm
  song_suggest = key.iloc[song_index].to_frame().columns.item()

  #return the song id to plug into the spotify appi and have song play
  return song_suggest

In [25]:
id_grab('dancing', 'abba')

'2aS8oHnf63s1vbWMSDLnCv'

The above function is almost done, here are the problems that need to be fixed:

1. the song_suggest var isn't grabbing the index value, which is what I wanted, it's grabbing the column titles. 
2. the "if title and artist" doesn't work atm because it doens't know what to do w/ title and artist. This shouldn't be far off, but it's 4am and i need to sleep. 

In [ ]:
# Needs work. Theoretically will take spotify song ID from id_grab and play song through spotify platform

def spotify(song):
  song_suggest = song
  song_id_uri = spotify:artist:song_suggest
  token = util.prompt_for_user_token(scope, client_id= client_id, client_secret = client_secret, redirect_uri='http://localhost:8881/')
  sp = spotipy.Spotify(auth=token)
  results = spotify.song(song_id_uri)
  while results['next']:
    results = spotify.next(results)
  


This spotify app is just a starting point, i think song_id_uri should read as 'artist:song:2l4Nmf7uR3wl4FZ6rre0Pp' for example. 

the token and sp should get the user to pull up their spotify app in window. That would be sick. the results i pulled off a random page. Basically this code may or may not be relevant. 

##Check Metrics

##Tune Model

## Communicate Results